In [10]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
%pip install kalshi


# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid
import os
from dotenv import load_dotenv

load_dotenv()  # Make sure this points to the correct path if .env isn't in the root directory


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [13]:
from KalshiClientsBaseV2 import ExchangeClient
import os
from dotenv import load_dotenv

load_dotenv()

# Set credentials
prod_email = os.getenv("EMAIL")
prod_password = os.getenv("PASSWORD")

demo_email = "friendsatchatmd@gmail.com"
demo_password = "TimmyNM12!"

# Set API base
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

# Initialize client (choose demo or prod)
exchange_client = ExchangeClient(exchange_api_base=demo_api_base, email=demo_email, password=demo_password)

# Test connection
print(exchange_client.get_exchange_status())

HttpError: HttpError(400 Bad Request)

In [ ]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {'limit':100,
                    'cursor':None, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker':None,
                    'max_close_ts':None, # pass in unix_ts
                    'min_close_ts':None, # pass in unix_ts
                    'status':None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

In [ ]:
# What are cursors and how do they work?
    
# The Cursor represents a pointer to the next page of records in the pagination.
# So this optional parameter, when filled, should be filled with the cursor string returned in a previous request to this end-point.
# Filling this would basically tell the api to get the next page containing the number of records passed on the limit parameter.
# On the other side not filling it tells the api you want to get the first page for another query.
# The cursor does not store any filters, so if any filter parameters like tickers, max_ts or min_ts were passed in the original query they must be passed again.

# Let's try it in action! Suppose we wanted to get the next 100 market objects...

market_params = {'limit':100,
                    'cursor':cursor, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker': None,
                    'max_close_ts': None, # pass in unix_ts
                    'min_close_ts': None, # pass in unix_ts
                    'status': None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']        

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in market_response payload:', markets_response['markets'][0]) # new markets!
print()
print('new cursor!', cursor)

In [ ]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response['markets'][5]['event_ticker']
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print('first market in event_response payload:', event_response['markets'][0])

In [ ]:
# Next, let's look at series level data by passing a series ticker to the get_series endpoint! 
series_ticker = event_response['event']['series_ticker']
series_params = {'series_ticker': series_ticker}
series_response = exchange_client.get_series(**series_params)

print('keys:', series_response.keys())
print()
print('series object:', series_response['series'])
print()

In [ ]:
# Next let's look at the recent market history for a market (ENDPOINT DOESN'T WORK)
# ticker = 'FED-25DEC-T4.25'

# market_history_params = {'ticker': ticker,
#                             'limit': 100,
#                             'cursor': None,
#                             'max_ts': None, # pass in unix_ts
#                             'min_ts': round(time.time()-1000000) # passing a recent unix_ts
#                                 }
# market_history_response = exchange_client.get_market_history(**market_history_params)

# print('keys:', market_history_response.keys())
# print()
# print('most recent market history object:', market_history_response['history'][-1])
# print()

# # and then also look at the most current view of the orderbook
# market_history_params = {'ticker':ticker,
#                             'depth': 30
#                                 }
# orderbook_response = exchange_client.get_orderbook(**market_history_params)

# print('keys:', orderbook_response.keys())
# print()
# print('orderbook object:', orderbook_response)
# print()
# Now let's suppose we wanted to place a trade on one of these markets... 
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

In [ ]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {'limit': None,
                        'cursor': None,
                        'settlement_status': None,
                        'ticker': None,
                        'event_ticker': None}

current_position = exchange_client.get_positions(**positions_params)
current_position

In [ ]:
# seems like some of your recent orders had been filled. To check on those we use the get_positions endpoint ...

fills_params = {'ticker':None,
                    'order_id':None,
                    'min_ts':None,
                    'max_ts':None,
                    'limit': None,
                    'cursor': None}

fills = exchange_client.get_fills(**fills_params)
fills

In [ ]:

# you may even want to check on some of your recent positions settled...

settlement_params = {'limit': None,
                        'cursor': None}

settlements = exchange_client.get_portfolio_settlements(**settlement_params)
settlements